This project completely uses Agno AI Agent. Although it has many features but it was often a struggle, it felt that using embedding independently would have been much easier. Also it's difficult to locate the updates attributes name everytime. Langchain has an upper hand over this feature.

In [77]:
%pip install pygithub lancedb sentence-transformers agno

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lancedb/__init__.py:220: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [78]:
from github import Github
from itertools import islice    #con of popular repos, they have many issues !!, so we trim to test 20 issues

In [ ]:
g=Github("your_github_token")

In [80]:
repo = g.get_repo("NVIDIA/physicsnemo")    

In [81]:
raw_issues = [
    {
        "number": issue.number,
        "title": issue.title,
        "body": issue.body or "",
        "url": issue.html_url
    }
    for issue in islice(repo.get_issues(state="all"), 20)
    if issue.pull_request is None
]

In [82]:
#loop to get all issues
texts = [f"{issue['title']} {issue['body']}" for issue in raw_issues]

In [83]:
#embedding
from agno.embedder.sentence_transformer import SentenceTransformerEmbedder

embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2")

In [84]:
from agno.vectordb.lancedb import LanceDb
from agno.agent import AgentKnowledge

In [ ]:
#lanceDB baby directly store with embedding model
vector_db = LanceDb(table_name="github-issues",embedder=embedder)

In [86]:
results = vector_db.search(query="login issue")

In [87]:
context = "\n\n".join([f"Issue {i+1}:\n{doc.content}" for i, doc in enumerate(results)])

In [88]:
prompt = f"""
You are a helpful assistant. Here are some GitHub issues:

{context}

Summarize the common problems or patterns described in these issues.
"""

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "your_api_key"

In [90]:
from agno.agent import Agent
from agno.models.google import Gemini  
from agno.tools.reasoning import ReasoningTools

agent = Agent(
    model=Gemini(id="models/gemini-2.0-flash-lite"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Identify important themes",
        "Group similar issues",
        "Highlight core problems in one line"
    ],
    markdown=True,
)

In [91]:
response = agent.run(prompt)

In [92]:
print("\n---- Gemini Summary ----\n")
print(response)


---- Gemini Summary ----

RunResponse(content="Here's a breakdown of the common problems and patterns across the provided GitHub issues:\n\n*   **Issue 1 & 2: Regression and Diffusion Model Errors after PR merge:** Both issues describe errors encountered in the CorrDiff example after a specific pull request. Issue 1 is a `TypeError` related to an unexpected keyword argument in `RegressionLoss`. Issue 2 is a `RuntimeError` related to a type mismatch during generation, potentially linked to changes in the PR. **Core Problem:** Errors introduced in the CorrDiff example after the specified PR, specifically impacting training and generation, potentially due to inconsistencies in the code or data types.\n\n*   **Issue 3: NaN values with `@StaticCaptureEvaluateNoGrad`:** This issue highlights a problem where using a specific decorator (`@StaticCaptureEvaluateNoGrad`) leads to the generation of NaN values during inference, while standard PyTorch inference does not. **Core Problem:** NaN value